In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from sodapy import Socrata

In order to evaluate the efficacy of different interventions in the fight against the opioid epidemic, I am reviewing prescription and enrollment information for Kentucky Medicaid. The data is available publicly through data.medicaid.gov and the Socrata client. SodaPy is the Python library that works with Socrata. 

I obtained 5 years of prescription data (2015-2019) and the corresponding enrollment. I also obtained a dataset of only the medications prescribed for substance use disorder treatment, including Naloxone (Narcan). 

I will review the data from each year 2015-2018 to create forecasts, then compare the forecasts to the actual prescription values in 2019. 

This is the basis of my evaluation of the impact of US DOJ interventions in KY starting in October, 2018. 

In [2]:
client = Socrata("data.medicaid.gov", None)
#code from the medicaid.gov website for using Socrata with Python

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.medicaid.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

rx_2015 = client.get("hse5-m4bk", state_code = 'KY', limit = 250000)
rx_2016 = client.get("dpqa-tc6u", state_code = 'KY', limit = 250000)
rx_2017 = client.get("qams-sami", state_code = 'KY', limit = 250000)
rx_2018 = client.get("ddz4-5k5v",state_code = 'KY', limit = 250000)
rx_2019 = client.get("8fjh-49cj", state_code = 'KY', limit = 250000)
rx_2020 = client.get("yuuq-gv5v", state_code = 'KY', limit = 250000)


rx_mat = client.get("3mnv-bath", state_code = 'KY', limit = 10000)
mcd_enrollment =client.get("nkdi-f9a2", limit = 100)


# Convert to pandas DataFrame
df_2020 = pd.DataFrame.from_records(rx_2020)
df_2019 = pd.DataFrame.from_records(rx_2019)
df_2018 = pd.DataFrame.from_records(rx_2018)
df_2017 = pd.DataFrame.from_records(rx_2017)
df_2016 = pd.DataFrame.from_records(rx_2016)
df_2015 = pd.DataFrame.from_records(rx_2015)
df_mat = pd.DataFrame.from_records(rx_mat)
df_enrollment =pd.DataFrame.from_records(mcd_enrollment)

Once all the data is obtained, check the shape of each year's data set, consolidate into 1 data frame for years 2015-2018, remove all rows with nulls in the 'number of prescriptions' field. 

In [3]:
df_2015.shape, df_2016.shape, df_2017.shape, df_2018.shape, df_2019.shape, df_mat.shape, df_enrollment.shape


((116873, 20),
 (117983, 17),
 (126557, 20),
 (130942, 20),
 (132900, 20),
 (262, 20),
 (66, 9))

In [4]:
df_2015.columns, df_2016.columns, df_2017.columns, df_2018.columns, df_2019.columns

(Index(['quarter', 'state_code', 'record_id', '_quarter_begin_date',
        '_latitude', 'product_code', 'package_size', '_quarter_begin',
        'non_medicaid_amount_reimbursed', 'suppression_used',
        'number_of_prescriptions', 'product_fda_list_name', 'labeler_code',
        'total_amount_reimbursed', 'units_reimbursed', 'period_covered',
        '_longitude', 'medicaid_amount_reimbursed', '_location', 'ndc'],
       dtype='object'),
 Index(['quarter', 'record_id', 'state_code', '_quarter_begin_date',
        'product_code', 'package_size', '_quarter_begin',
        'non_medicaid_amount_reimbursed', 'number_of_prescriptions',
        'product_fda_list_name', 'labeler_code', 'total_amount_reimbursed',
        'units_reimbursed', 'period_covered', 'medicaid_amount_reimbursed',
        '_location', 'ndc'],
       dtype='object'),
 Index(['quarter', 'state_code', 'record_id', '_quarter_begin_date',
        '_latitude', 'product_code', 'package_size', '_quarter_begin',
        'no

Define the fields that are consistent across all prescription data and identify features to use in model

In [5]:
keepers = ['quarter', 'state_code', 
        'product_code', 'package_size', '_quarter_begin', 'number_of_prescriptions',
        'product_fda_list_name', 'labeler_code', 'total_amount_reimbursed',
        'units_reimbursed', 'period_covered', 'ndc']

Use concatenate to create one data frame for years 2015-2018, and drop any rows with NaN in number_of_prescriptions

In [69]:
rx_df = pd.concat([df_2015[keepers],df_2016[keepers], df_2017[keepers], df_2018[keepers]] )

In [70]:
rx_df.dropna(subset = ['number_of_prescriptions'], inplace =True)

In [71]:
rx_df.shape

(203597, 12)

In [72]:
rx_df.head()

,quarter,state_code,product_code,package_size,_quarter_begin,number_of_prescriptions,product_fda_list_name,labeler_code,total_amount_reimbursed,units_reimbursed,period_covered,ndc
0,2,KY,0167,58,4/1,1499,METOPROLOL,57664,6701.03,84920,2015,57664016758
1,4,KY,0759,20,10/1,58,ATENOLOL,51079,72.91,511,2015,51079075920
2,2,KY,0117,04,4/1,2038,LACTATED R,00338,25361.71,2136666,2015,00338011704
3,3,KY,0923,55,7/1,47,PROTONIX I,00008,457.27,113,2015,00008092355
4,3,KY,0044,03,7/1,60,REBIF,44087,323096.78,250.5,2015,44087004403


to limit the prescription data to only opioid drugs, I'm using the OpenFDA API to query the FDA NDC drug database. This database is updated daily with the latest information from the FDA about all products with an NDC (national drug code) identifier. The API documentation indicates the search is limited to a max of 1000 rows, and according to the metadata, there are 1751 opioid drugs in the database. 2 pulls with a maximum limit of 1000 rows each should provide all the opioids in the database 
Documentation for the API is here: https://open.fda.gov/apis/

In [9]:
data1 = requests.get('https://api.fda.gov/drug/ndc.json?search=pharm_class:"Opioid"&limit=1000').json()
data2 = requests.get('https://api.fda.gov/drug/ndc.json?search=pharm_class:"Opioid"&limit=1000&skip=1000').json()


In [35]:
#code assistance from James

inner = data1['results'] + data2['results']
fda_df = pd.DataFrame.from_records(inner)
fda_df['route'] = fda_df['route'].str[0]

fda_df = fda_df[['product_ndc','generic_name','dea_schedule','brand_name','active_ingredients','route','pharm_class']]


1753    {'is_original_packager': [True], 'spl_set_id':...
1754    {'is_original_packager': [True], 'spl_set_id':...
1755    {'is_original_packager': [True], 'spl_set_id':...
1756    {'is_original_packager': [True], 'spl_set_id':...
1757    {'is_original_packager': [True], 'spl_set_id':...
Name: openfda, dtype: object

Splitting the list into 5 smaller dataframes based on DEA drug class. Most, but not all, partial opioid agonists used in medication-assisted substance abuse treatment are classified as Schedule III prescription medications. 
This may be useful if I decide to use the DEA class as a feature or summarize total rx by schedule. Also a quick way to see what meds are in which class using value-count

In [63]:
CII = fda_df.loc[fda_df['dea_schedule']== 'CII']


CIII = fda_df.loc[fda_df['dea_schedule'] == 'CIII']



CIV = fda_df.loc[fda_df['dea_schedule'] == 'CIV']



CV = fda_df.loc[fda_df['dea_schedule'] == 'CV']


array(['Meperidine Hydrochloride', 'hydromorphone hydrochloride',
       'MORPHINE SULFATE', 'HYDROMORPHONE HYDROCHLORIDE',
       'oxycodone hydrochloride', 'Fentanyl Citrate', 'Morphine Sulfate',
       'Hydromorphone Hydrochloride', 'SUFENTANIL CITRATE',
       'Hydrocodone Bitartrate and Acetaminophen',
       'Oxycodone and Acetaminophen', 'Methadone Hydrochloride',
       'Oxycodone Hydrochloride', 'OXYMORPHONE HYDROCHLORIDE',
       'oxycodone and acetaminophen tablets',
       'oxycodone and acetaminophen',
       'oxycodone hydrochloride and acetaminophen', 'FENTANYL CITRATE',
       'METHADONE HYDROCHLORIDE',
       'hydrocodone bitartrate and homatropine methylbromide',
       'Hydromorphone hydrochloride', 'Codeine Sulfate',
       'Levorphanol Tartrate', 'OXYCODONE HYDROCHLORIDE', 'OXYCODONE',
       'tapentadol hydrochloride',
       'Hydrocodone Polistirex and Chlorpheniramine Polistirex',
       'FENTANYL', 'fentanyl', 'morphine sulfate',
       'Oxycodone Hydchloride',

Only methadone, butorphanol tartrate, and buprenorphine-containing medications are acceptable forms of medicated-assisted treatment for substance use disorder covered by Medicaid in KY (there are other non-opioid medications like Wellbutrin approved for substance abuse treatment, but I've limited my dataframe to only opioid medications). Create a dummy column for MAT - '1' if lowercase generic name contains methadone, butorphanol, or buprenorphine; '0' if not.

Likewise, create a dummy column for Opioid - '1' if lowercase generic name contains one of the base opioids. I started with trying to match NDC codes, but discovered that the current directory does not include recently retired NDC's. Out of 203k transaction records, 6k registered as opioids when I matched on NDC's. With the dummy column based on name, I also capture opioid drugs historically prescribed under a now-expired NDC. 

Using this example from StackOverflow as my basis - https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o?rq=1 and assistance from Alex getting my syntax correct.

In [99]:
def mat(value):
    
    if 'buprenorph' in value.lower():
        return 1
    elif 'methadone' in value.lower():
        return 1
    elif 'butorphano' in value.lower():
        return 1
    else:
        return 0
    


In [117]:
def opioid(value):
    if 'oxycodone' in value.lower():
        return 1
    elif 'fentanyl' in value.lower():
        return 1
    elif 'morphine' in value.lower():
        return 1
    elif 'buprenorph' in value.lower():
        return 1
    elif 'meperidine' in value.lower():
        return 1
    elif 'hydromorph' in value.lower():
        return 1
    elif 'methadone' in value.lower():
        return 1
    elif 'hydrocodon' in value.lower():
        return 1
    elif 'tapentadol' in value.lower():
        return 1
    elif 'oxymorphon' in value.lower():
        return 1
    elif 'codeine' in value.lower():
        return 1
    elif 'tramadol' in value.lower():
        return 1
    elif 'pentazocin' in value.lower():
        return 1
    elif 'butorphano' in value.lower():
        return 1
    elif 'eluxadolin' in value.lower():
        return 1
    elif 'sufentanil' in value.lower():
        return 1
    elif 'demerol' in value.lower():
        return 1
    elif 'remifentan' in value.lower():
        return 1
    elif 'alfentanil' in value.lower():
        return 1
    elif 'levorphano' in value.lower():
        return 1
    elif 'lortab' in value.lower():
        return 1
    elif 'demerol' in value.lower():
        return 1
    elif 'percocet' in value.lower():
        return 1
    elif 'endocet' in value.lower():
        return 1
    elif 'oxycontin' in value.lower():
        return 1
    elif 'hydromet' in value.lower():
        return 1
    elif 'nucynta' in value.lower():
        return 1
    elif 'dilaudid' in value.lower():
        return 1
    elif 'zohydro' in value.lower():
        return 1
    elif 'xtampza' in value.lower():
        return 1
    elif 'duramorph' in value.lower():
        return 1
    else:
        return 0
    

In [118]:
mat('Buprenorphine Hydrocholoride')

1

In [119]:
rx_df['mat'] = rx_df['product_fda_list_name'].apply(lambda x: mat(x))

In [120]:
rx_df['opioid']=rx_df['product_fda_list_name'].apply(lambda x: opioid(x))

In [121]:
rx_df['opioid'].value_counts()

0    197380
1      6217
Name: opioid, dtype: int64

Harmonize NDC format across both data sources by stripping the hyphen from product_ndc in the FDA dataset and dropping the last 2 digits in the ndc field in the Medicaid data

In [122]:
fda_df.columns

Index(['product_ndc', 'generic_name', 'labeler_name', 'dea_schedule',
       'brand_name', 'active_ingredients', 'finished', 'packaging',
       'listing_expiration_date', 'openfda', 'marketing_category',
       'dosage_form', 'spl_id', 'product_type', 'route',
       'marketing_start_date', 'product_id', 'application_number',
       'brand_name_base', 'pharm_class', 'marketing_end_date',
       'brand_name_suffix', 'label_code', 'prod_code', 'ndc', 'mat'],
      dtype='object')

Medicaid pads their NDC numbers. Example- NDC 0406-0540 is 004060540; 76420-127 is 764200127. In other words, the digits to the left of the hash are converted to a 5-digit 'labeler code' and the digits to the right of the hash are converted to a 4-digit 'product code'. 

In [123]:
fda_df['label_code']=fda_df['product_ndc'].str.split('-').str[0].str.zfill(5).astype("string")

fda_df['prod_code']=fda_df['product_ndc'].str.split('-').str[1].str.zfill(4).astype("string")

fda_df['ndc'] = fda_df['label_code']+fda_df['prod_code']

fda_df.drop_duplicates(subset = ['product_ndc'], inplace = True)

In [124]:
fda_df.tail()

,product_ndc,generic_name,labeler_name,dea_schedule,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,...,product_id,application_number,brand_name_base,pharm_class,marketing_end_date,brand_name_suffix,label_code,prod_code,ndc,mat
1753,0406-0527,methadone hydrochloride,SpecGx LLC,CII,METHADOSE,"[{'strength': '10 mg/mL', 'name': 'METHADONE H...",True,"[{'marketing_start_date': '19730314', 'package...",20211231,"{'is_original_packager': [True], 'spl_set_id':...",...,0406-0527_bdce787c-0fb4-4006-9165-ced25f38b8c1,NDA017116,METHADOSE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN,00406,0527,004060527,1
1754,0406-0540,Methadone Hydrochloride,SpecGx LLC,CII,METHADOSE DISPERSIBLE,"[{'strength': '40 mg/1', 'name': 'METHADONE HY...",True,"[{'marketing_start_date': '19930429', 'package...",20211231,"{'is_original_packager': [True], 'spl_set_id':...",...,0406-0540_481e16e5-e3c5-4d09-95a3-0f7a0233d6f0,ANDA074184,METHADOSE DISPERSIBLE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN,00406,0540,004060540,1
1755,0406-1170,naltrexone hydrochloride,SpecGx LLC,NaN,NALTREXONE HYDROCHLORIDE,"[{'strength': '50 mg/1', 'name': 'NALTREXONE H...",True,"[{'marketing_start_date': '20020322', 'package...",20211231,"{'is_original_packager': [True], 'spl_set_id':...",...,0406-1170_f7ac0547-6c8d-4a03-91f4-2c37215554ae,ANDA076264,NALTREXONE HYDROCHLORIDE,"[Opioid Antagonist [EPC], Opioid Antagonists [...",NaN,NaN,00406,1170,004061170,0
1756,0406-1009,Oxymorphone Hydrochloride,SpecGx LLC,CII,OXYMORPHONE HYDROCHLORIDE,"[{'strength': '5 mg/1', 'name': 'OXYMORPHONE H...",True,"[{'marketing_end_date': '20210131', 'marketing...",NaN,"{'is_original_packager': [True], 'spl_set_id':...",...,0406-1009_0c2957c6-5714-416c-9afb-98ee965950f7,ANDA202321,OXYMORPHONE HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",20210131,NaN,00406,1009,004061009,0
1757,0406-1010,Oxymorphone Hydrochloride,SpecGx LLC,CII,OXYMORPHONE HYDROCHLORIDE,"[{'strength': '10 mg/1', 'name': 'OXYMORPHONE ...",True,"[{'marketing_end_date': '20210531', 'marketing...",NaN,"{'is_original_packager': [True], 'spl_set_id':...",...,0406-1010_0c2957c6-5714-416c-9afb-98ee965950f7,ANDA202321,OXYMORPHONE HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",20210531,NaN,00406,1010,004061010,0


In [125]:
rx_df['ndc'] = rx_df['ndc'].str[:9]

Create a smaller data frame of just 'ndc', 'mat', and 'generic_name' from fda_df. Use merge to join rx_df to opioid_df using 'ndc'. The 'MAT' field in the fda_df serves as an indicator if the NDC in rx_df is an opioid and if so, if it is used for medication-assisted therapy

In [126]:
opioid_df = fda_df[['ndc','mat','generic_name']]

opioid_df.mat.value_counts()

merged_df= rx_df.merge(opioid_df, how = "left", on = "ndc")

merged_df.info()




<class 'pandas.core.frame.DataFrame'>
Int64Index: 203597 entries, 0 to 203596
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   quarter                  203597 non-null  object 
 1   state_code               203597 non-null  object 
 2   product_code             203597 non-null  object 
 3   package_size             203597 non-null  object 
 4   _quarter_begin           203597 non-null  object 
 5   number_of_prescriptions  203597 non-null  object 
 6   product_fda_list_name    203597 non-null  object 
 7   labeler_code             203597 non-null  object 
 8   total_amount_reimbursed  203597 non-null  object 
 9   units_reimbursed         203597 non-null  object 
 10  period_covered           203597 non-null  object 
 11  ndc                      203597 non-null  object 
 12  mat_x                    203597 non-null  int64  
 13  opioid                   203597 non-null  int64  
 14  mat_

As a way to cross-check that all opioid containing prescriptions are labeled as such, look for rows with a value of '0' for 'mat_y' (which means that it was an opioid in the NDC list from the FDA) and '0' for 'Opioid' which means that it did not match any of the base ingredient names passed to the prescription list.

In [127]:
part = merged_df[merged_df['generic_name'].isnull()]

In [128]:
part[(part['opioid'] == 1)]

,quarter,state_code,product_code,package_size,_quarter_begin,number_of_prescriptions,product_fda_list_name,labeler_code,total_amount_reimbursed,units_reimbursed,period_covered,ndc,mat_x,opioid,mat_y,generic_name
132,3,KY,0229,01,7/1,238,HYDROCODON,51862,6427.34,22498,2015,518620229,0,1,NaN,NaN
175,3,KY,2605,05,7/1,40,HYDROCODON,00591,988.06,2553,2015,005912605,0,1,NaN,NaN
200,3,KY,2612,05,7/1,137,HYDROCODON,00591,3232.17,10700,2015,005912612,0,1,NaN,NaN
228,3,KY,0227,01,7/1,69,HYDROCODON,51862,716.38,3265,2015,518620227,0,1,NaN,NaN
241,2,KY,6070,25,4/1,542,MORPHINE S,00641,1810.24,596,2015,006416070,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203203,1,KY,0228,01,1/1,60,HYDROCODON,51862,203.12,2560,2018,518620228,0,1,NaN,NaN
203256,2,KY,0532,10,4/1,55,HYDROCODON,63739,28.21,12,2018,637390532,0,1,NaN,NaN
203304,2,KY,0601,01,4/1,111,HYDROCODON,68084,221.22,133,2018,680840601,0,1,NaN,NaN
203305,3,KY,3356,01,7/1,12,HYDROMORPH,00409,32.43,26,2018,004093356,0,1,NaN,NaN


In [129]:
play = merged_df[(merged_df['opioid'] == 0)]

In [130]:
q = play[(play['mat_y']==0)]

In [131]:
q['product_fda_list_name'].unique()

array(['NALTREXONE', 'ACETAMINOP', 'PROMETHAZI', 'BUTALB-CAF',
       'LOPERAMIDE', 'BUTALBITAL', 'TYLENOL-CO', 'RELISTOR', 'NALOXONE H',
       'NALBUPHINE', 'BUTALB-ACE', 'ASCOMP WIT', 'MOVANTIK', 'NARCAN',
       'VIBERZI', 'UNKNOWN', 'EVZIO', 'CONTRAVE', 'SYMPROIC'],
      dtype=object)

Some of the remaining Rx weren't recognized because the brand name was prescribed. Some aren't recognized because they are medications containing an opioid, but it is not the primary ingredient and therefore not listed in the 10-character 'product_fda_list_name' field. 